# Project: Danny's Diner

## Problem Statement

* Danny’s Diner is in need of your assistance to help the restaurant stay afloat. The restaurant has captured some very basic data from their few months of operation.
* Danny wants to use the data to answer a few simple questions about his customers. He wants information about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

### Connect to SQL

In [2]:
%%capture
%load_ext sql
%sql sqlite://

### Sales Table

* The sales table captures all customer_id level purchases with a corresponding order_date and product_id information for when and what menu items were ordered.

In [3]:
%%sql
CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');

 * sqlite://
Done.
15 rows affected.


[]

### Menu Table

The menu table maps the product_id to the actual product_name and price of each menu item.

In [4]:
%%sql
CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
  

 * sqlite://
Done.
3 rows affected.


[]

### Members Table

* The members table captures the join_date when a customer_id joined the beta version of the Danny’s Diner loyalty program.

In [5]:
%%sql
CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

 * sqlite://
Done.
2 rows affected.


[]

### 1. What is the total amount each customer spent at the restaurant?

In [6]:
%%sql
SELECT
    sales.customer_id,
    SUM(price) as total_spent
FROM sales
JOIN menu
ON sales.product_id = menu.product_id
GROUP BY 1

 * sqlite://
Done.


customer_id,total_spent
A,76
B,74
C,36


- Customer A spent 76 dollars, customer B spent 74 dollars and customer C spent 36 dollars.

### 2. How many days has each customer visited the restaurant?

In [7]:
%%sql
SELECT 
    customer_id,
    COUNT(DISTINCT order_date) as days_visited
FROM sales
GROUP BY 1

 * sqlite://
Done.


customer_id,days_visited
A,4
B,6
C,2


* Customer A visited on 4 days, customer B visited 6 days and customer C visited only twice.

### 3. What was the first item from the menu purchased by each customer?

In [8]:
%%sql
SELECT 
    sales.customer_id,
    menu.product_name as first_item
FROM sales
JOIN menu 
ON sales.product_id = menu.product_id
WHERE sales.order_date = (
  SELECT MIN(order_date)
  FROM sales
)
GROUP BY 1

 * sqlite://
Done.


customer_id,first_item
A,sushi
B,curry
C,ramen


* Customer A ordered sushi first, customer B ordered curry first, and customer C ordered ramen first.

### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [9]:
%%sql
SELECT
    menu.product_name,
    COUNT(sales.product_id) as times_purchased
FROM sales
JOIN menu
ON sales.product_id = menu.product_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1

 * sqlite://
Done.


product_name,times_purchased
ramen,8


* The most purchased item on the menu was Ramen which was purchased 8 times.

### 5. Which item was the most popular for each customer?

In [10]:
%%sql
WITH t1 AS (
SELECT
    sales.customer_id,
    menu.product_name,
    COUNT(menu.product_id) as purchase_count
FROM sales
JOIN menu
ON sales.product_id = menu.product_id
GROUP BY 1, 2
ORDER BY 1, 3 DESC
)

SELECT 
    customer_id,
    menu.product_name,
    MAX(purchase_count) as most_purchased
FROM t1
JOIN menu
ON t1.product_name = menu.product_name
GROUP BY 1

 * sqlite://
Done.


customer_id,product_name,most_purchased
A,ramen,3
B,sushi,2
C,ramen,3


* The most popular item for customers A and C was Ramen with 3 orders each while customer B ordered Sushi the most with 2 orders. 

### 6. Which item was purchased first by the customer after they became a member?

In [11]:
%%sql
WITH first_purchase AS (
SELECT 
    sales.customer_id, 
    MIN(sales.order_date) AS first_purchase_date
FROM sales
JOIN members 
ON sales.customer_id = members.customer_id
WHERE sales.order_date >= members.join_date
GROUP BY sales.customer_id
)

SELECT 
    sales.customer_id, 
    sales.order_date, 
    menu.product_name
FROM first_purchase
JOIN sales
ON first_purchase.customer_id = sales.customer_id AND first_purchase.first_purchase_date = sales.order_date
JOIN menu
ON sales.product_id = menu.product_id
ORDER BY 1

 * sqlite://
Done.


customer_id,order_date,product_name
A,2021-01-07,curry
B,2021-01-11,sushi


* Customer A ordered Curry first after they became a member while Customer B ordered Sushi.

### 7. Which item was purchased just before the customer became a member?

In [23]:
%%sql
WITH last_purchase AS (
    SELECT 
        sales.customer_id, 
        MAX(sales.order_date) AS last_purchase_date
    FROM sales
    JOIN members 
    ON sales.customer_id = members.customer_id
    WHERE sales.order_date < members.join_date
    GROUP BY sales.customer_id
)

SELECT
    sales.customer_id,
    sales.order_date,
    product_name
FROM last_purchase
JOIN sales
ON last_purchase.customer_id = sales.customer_id AND last_purchase.last_purchase_date = sales.order_date
JOIN menu
ON sales.product_id = menu.product_id

 * sqlite://
Done.


customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-04,sushi


* The last order Customer A had before becoming a member was Curry.
* The last order Customer B had before becoming a member was Sushi.

### 8. What is the amount spent for each member before they became a member?

In [33]:
%%sql
SELECT
    sales.customer_id,
    SUM(price) as total_spent
FROM sales
JOIN menu
ON sales.product_id = menu.product_id
JOIN members 
ON sales.customer_id = members.customer_id
WHERE order_date < join_date
GROUP BY 1

 * sqlite://
Done.


customer_id,total_spent
A,25
B,40


* Customer A spent 25 dollars prior to becoming a member while Customer C spent 40 dollars.

### 9. If each 1 dollar spent equates to 10 points and sushi has a 2x points multiplier - How many points would each customer have?

In [39]:
%%sql
SELECT
    sales.customer_id,
    SUM(CASE WHEN menu.product_name = 'sushi' THEN (menu.price * 20)
        ELSE (menu.price * 10)
        END) as points_earned
FROM sales
JOIN menu
ON sales.product_id = menu.product_id
GROUP BY 1
ORDER BY 2 DESC


 * sqlite://
Done.


customer_id,points_earned
B,940
A,860
C,360


* Customer B has earned the most points with 940, followed by Customer A with 860 points and Customer C with 360 points.

### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [137]:
%%sql
SELECT 
    sales.customer_id, 
    SUM(CASE WHEN sales.order_date >= members.join_date AND sales.order_date < (members.join_date + 'INTERVAL 7 days')
    OR (menu.product_name = 'sushi') THEN (menu.price * 20)
    ELSE (menu.price * 10)
    END) as points_earned
FROM sales
JOIN menu
ON sales.product_id = menu.product_id
JOIN members
ON sales.customer_id = members.customer_id
WHERE sales.order_date <= '2021-01-31'
GROUP BY 1


 * sqlite://
Done.


customer_id,points_earned
A,860
B,820


* Customer A would have 1370 points at the end of January and Customer B would have 820 points.

### Bonus Question # 1
* Create a table with the columns customer_id, order_date, product_name, and price.
* Add a new column called member. The value of member should be 'N' if the order took place before a customer became a member and 'Y' if the order took place after the customer became a member.

In [93]:
%%sql
SELECT
    sales.customer_id,
    sales.order_date,
    menu.product_name,
    menu.price,
    (CASE WHEN sales.order_date >= members.join_date THEN 'Y'
    ELSE 'N'
    END) as member
FROM sales
JOIN menu
ON sales.product_id = menu.product_id
LEFT JOIN members
ON sales.customer_id = members.customer_id

 * sqlite://
Done.


customer_id,order_date,product_name,price,member
A,2021-01-01,sushi,10,N
A,2021-01-01,curry,15,N
A,2021-01-07,curry,15,Y
A,2021-01-10,ramen,12,Y
A,2021-01-11,ramen,12,Y
A,2021-01-11,ramen,12,Y
B,2021-01-01,curry,15,N
B,2021-01-02,curry,15,N
B,2021-01-04,sushi,10,N
B,2021-01-11,sushi,10,Y


### Bonus Question # 2
* Take the table you created in Bonus Question # 1
* Add a new column called ranking. The value of ranking should be null if the order took place before a customer became a member. If the order took place after the customer became a member, then add a rank to each order. For example, the first order that took place after a customer became a member should be 1. If two orders took place on the same day, they should have the same ranking.

In [130]:
%%sql
WITH cte AS (
SELECT
    sales.customer_id,
    sales.order_date,
    menu.product_name,
    menu.price,
    (CASE WHEN sales.order_date >= members.join_date THEN 'Y'
    ELSE 'N'
    END) as member
FROM sales
JOIN menu
ON sales.product_id = menu.product_id
LEFT JOIN members
ON sales.customer_id = members.customer_id
)

SELECT
    *,
    (CASE WHEN member = 'N' THEN 'null'
     WHEN member = 'Y' THEN DENSE_RANK() OVER (PARTITION BY customer_id, member ORDER BY order_date)
     END) as ranking
FROM cte
ORDER BY 1

 * sqlite://
Done.


customer_id,order_date,product_name,price,member,ranking
A,2021-01-01,sushi,10,N,null
A,2021-01-01,curry,15,N,null
A,2021-01-07,curry,15,Y,1
A,2021-01-10,ramen,12,Y,2
A,2021-01-11,ramen,12,Y,3
A,2021-01-11,ramen,12,Y,3
B,2021-01-01,curry,15,N,null
B,2021-01-02,curry,15,N,null
B,2021-01-04,sushi,10,N,null
B,2021-01-11,sushi,10,Y,1
